In [3]:
# Course : CS 513 - Knowledge Discovery and Data Mining
# Group Members: Branden Bulatao, Joseph Faustino, Natalie Fortes, Isabel Sutedjo
# Id : 20005971, 20006114, 20006007, 20006618
# Purpose : Music Happiness Predictor - Predicts the happiness of music tracks based on various features.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("dataset.csv")

# Focus on top 5 genres
top_genres = df['track_genre'].value_counts().nlargest(5).index
df = df[df['track_genre'].isin(top_genres)]

# Select relevant features
features = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'valence', 'tempo']
X = df[features]
y = df['track_genre']

# Drop missing values
X = X.dropna()
y = y[X.index]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Base estimator (Decision Tree with max depth to prevent overfitting)
base_tree = DecisionTreeClassifier(criterion='entropy', max_depth=5)

# Boosted model (C5.0-like)
model = AdaBoostClassifier(estimator=base_tree, n_estimators=50, learning_rate=1.0)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("C5.0-style (Boosted) Decision Tree Report:")
print(classification_report(y_test, y_pred))

C5.0-style (Boosted) Decision Tree Report:
                   precision    recall  f1-score   support

         acoustic       0.68      0.60      0.64       316
              pop       0.61      0.63      0.62       309
        power-pop       0.57      0.49      0.53       284
progressive-house       0.73      0.75      0.74       301
        punk-rock       0.43      0.52      0.47       290

         accuracy                           0.60      1500
        macro avg       0.60      0.60      0.60      1500
     weighted avg       0.61      0.60      0.60      1500

